In [169]:
import pandas as pd
import numpy as np

import yfinance as yf
import finnhub

import time
from datetime import datetime as dt, timedelta
from dateutil.relativedelta import relativedelta

import json
import requests
from bs4 import BeautifulSoup

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from tqdm import tqdm

from sklearn.preprocessing import LabelEncoder

In [59]:
pd.set_option('display.max_rows', 50)

# Read in the Tickers

In [2]:
stocks = pd.read_excel('Data/universe.xlsx')['symbol']

# Fetching the Data

### Stock Prices

In [3]:
# Convert pd Series of strings into one long string (that the format yf wants)
string_format_stocks = stocks.str.cat(sep=' ')

# Fetch the price data
prices = yf.download(string_format_stocks, start='2021-01-01', end=dt.today().strftime('%Y-%m-%d'))

# Extract only the adj close data
adj_close = prices['Adj Close']
volume = prices['Volume']

[*********************100%***********************]  110 of 110 completed


In [4]:
prices.columns = prices.columns.swaplevel(0, 1)
prices.sort_index(axis=1, level=0, inplace=True)

### Industry Breakdown

In [5]:
# Initialize finhub client
finnhub_client = finnhub.Client(api_key="ccn3d6iad3i1nkrepma0ccn3d6iad3i1nkrepmag")

In [6]:
# Fetch the company data from finhub
industries = []

for count, ticker in enumerate(stocks):

    try:
        tickerdata = finnhub_client.company_profile2(symbol=ticker)
        industries.append(tickerdata['finnhubIndustry'])
    except Exception:
        industries.append(np.nan)

In [7]:
# Count the number of occurences of each industry
industries = np.array(industries)
unique, counts = np.unique(industries, return_counts=True)

# Calculate the precentage of industries
percentages = np.round(counts/np.sum(counts) * 100, 2)
print('Breakdown by Indsutry:')
print()
industries_dict = dict(zip(unique, percentages))
display(pd.DataFrame.from_dict(industries_dict, orient='index', columns=['Percentage of Total']))

Breakdown by Indsutry:



,Percentage of Total
Banking,0.91
Beverages,0.91
Biotechnology,0.91
Building,1.82
Chemicals,0.91
Commercial Services & Supplies,1.82
Consumer products,0.91
Distributors,0.91
Electrical Equipment,0.91
Energy,1.82


### Read in Previous Week Data

In [8]:
prev_general_market_news = pd.read_csv('Data/general_market_news.csv', index_col=0)
prev_financial_news = pd.read_csv('Data/financial_news.csv', index_col=0)
prev_ticker_news = pd.read_csv('Data/ticker_news.csv', index_col=0)

In [9]:
# Get the latest date from each file to start fetching new news articles from that date
general_market_news_latest_date = pd.to_datetime(prev_general_market_news['Date']).max().date()
financial_news_latest_date = pd.to_datetime(prev_financial_news['Date']).max().date()
ticker_news_latest_date = pd.to_datetime(prev_ticker_news['Date']).max().date()

# Convert datetime.date to datetime.datetime
general_market_news_latest_date = dt.combine(general_market_news_latest_date, dt.min.time())
financial_news_latest_date = dt.combine(financial_news_latest_date, dt.min.time())
ticker_news_latest_date = dt.combine(ticker_news_latest_date, dt.min.time())

# Fetch the data until this day
END_DATE_general = general_market_news_latest_date
END_DATE_financial = financial_news_latest_date
END_DATE_ticker = ticker_news_latest_date

### Market News

In [11]:
market_news = finnhub_client.general_news('general', min_id=0)

In [12]:
# Parse the news articles
parsed_market_news = []

for article in market_news:
    headline = article['headline']
    summary = article['summary']
    date = dt.fromtimestamp(article['datetime'])
    
    parsed_market_news.append([date, headline, summary])

In [13]:
# Convert to DataFrame
columns = ['Date', 'Headline', 'Summary']
general_market_news = pd.DataFrame(parsed_market_news, columns=columns)

## Alpha Vantage

In [14]:
ALPHA_VANTAGE_KEY = '4GIL4K9ZN1NWE26G'

In [67]:
# Create a list of dates with a weekly frequency
dates_timestamps = pd.date_range(start=END_DATE_financial, end=dt.today(), freq='W').tolist()
dates = list(map(dt.date, dates_timestamps))

In [68]:
# Helper function that extracts needed info from json object more efficiently
def extract_info_vantage(article):
    time_published = dt.strptime(article['time_published'][:8], '%Y%m%d')
    sentiment_score = article['overall_sentiment_score']
    sentiment_label = article['overall_sentiment_label']
    return [time_published, sentiment_score, sentiment_label]

# This is the response from the API when no articles are found
empty_response = {'Information': 'No articles found. Please adjust the time range or refer to the API documentation https://www.alphavantage.co/documentation#newsapi and try again.'}

In [69]:
# Fetch the following news

financial_market_news = []
economy_fiscal_news = []
economy_monetary_news = []
economy_macro_news = []

time_from = dates[0].strftime('%Y%m%d') + 'T0000' # start with the first date in the list

for date in dates[1:]:
    
    time_to = date.strftime('%Y%m%d') + 'T0000' # update the time_to

    # financial market news

    topic = 'financial_markets'

    url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&topics=' + topic \
            + '&time_from=' + time_from + '&time_to=' + time_to + '&limit=200&apikey=' + ALPHA_VANTAGE_KEY 
    result = requests.get(url)
    data = result.json()

    if data != empty_response:
        financial_market_news.extend(list(map(extract_info_vantage, data['feed']))) # add data to the list

    # Economy - Fiscal Policy news economy_fiscal

    topic = 'economy_fiscal'

    url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&topics=' + topic \
            + '&time_from=' + time_from + '&time_to=' + time_to + '&limit=200&apikey=' + ALPHA_VANTAGE_KEY 
    result = requests.get(url)
    data = result.json()

    if data != empty_response:
        economy_fiscal_news.extend(list(map(extract_info_vantage, data['feed']))) # add data to the list

    # Economy - Monetary Policy

    topic = 'economy_monetary'

    url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&topics=' + topic \
            + '&time_from=' + time_from + '&time_to=' + time_to + '&limit=200&apikey=' + ALPHA_VANTAGE_KEY 
    result = requests.get(url)
    data = result.json()

    if data != empty_response:
        economy_monetary_news.extend(list(map(extract_info_vantage, data['feed']))) # add data to the list

    # Economy - Macro/Overall

    topic = 'economy_macro'

    url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&topics=' + topic \
            + '&time_from=' + time_from + '&time_to=' + time_to + '&limit=200&apikey=' + ALPHA_VANTAGE_KEY 
    result = requests.get(url)
    data = result.json()

    if data != empty_response:
        economy_macro_news.extend(list(map(extract_info_vantage, data['feed']))) # add data to the list
    
    # now the time_from is the previous time to, so looking at the following week articles
    time_from = time_to 
    
    # unpaid subscription allows 5 calls per minute
    time.sleep(60)

In [70]:
# Convert lists of data to DataFrame objects
columns = ['Date', 'Sentiment Score', 'Sentiment Label']

financial_market_news_df = pd.DataFrame(financial_market_news, columns=columns)
financial_market_news_df['Topic'] = 'financial_market_news'                       # specify the topic of the df

economy_fiscal_news_df = pd.DataFrame(economy_fiscal_news, columns=columns)
economy_fiscal_news_df['Topic'] = 'economy_fiscal_news'                           # specify the topic of the df

economy_monetary_news_df = pd.DataFrame(economy_monetary_news, columns=columns)
economy_monetary_news_df['Topic'] = 'economy_monetary_news'                       # specify the topic of the df

economy_macro_news_df = pd.DataFrame(economy_macro_news, columns=columns)
economy_macro_news_df['Topic'] = 'economy_macro_news'                             # specify the topic of the df

# Concatenate these dfs into one long df
financial_news = pd.concat([financial_market_news_df, economy_fiscal_news_df, economy_monetary_news_df, economy_macro_news_df])

# Reset index (otherwise each index is repeated 4 times)
financial_news.reset_index(drop=True, inplace=True)

## Polygon.io

In [19]:
POLYGON_KEY = 'Chz4dhUuzmumD5YcQeSpI7M_JgItlGJc'

In [20]:
# Helper function that gets the dates of fetched news articles
def get_dates(dic):
    return dic['published_utc'][:10]

# Helper function that extracts needed info from json object more efficiently
def extract_info_polygon(article):
    time_published = article['published_utc'][:10]
    title = article['title']
    try:
        description = article['description']
    except:
        description = np.nan        
    return [time_published, title, description]

In [21]:
# Fetch news per ticker
request_counter = 0
news = {}

for num, ticker in enumerate(stocks):
    
    print(ticker, num)
    
    TICKER = ticker
    DATE = dt.today().strftime('%Y-%m-%d')
    
    news[TICKER] = []
    
    while dt.strptime(DATE, '%Y-%m-%d') > END_DATE_ticker:
        
        print('running for date:', DATE)
                        
        # Unpaid subscription allows 5 calls per minute
        if request_counter == 5:
            time.sleep(60)
            request_counter = 0
        
        # Fetch the news articles
        result = requests.get('https://api.polygon.io/v2/reference/news?order=desc&ticker=' + TICKER + 
                              '&published_utc.lte=' + DATE + '&limit=1000&apiKey=' + POLYGON_KEY)
        all_articles = json.loads(result.text)['results']
        
        request_counter += 1
        
        # Append needed info from the json object to a list
        news[ticker].extend(list(map(extract_info_polygon, all_articles)))
            
        # Get the latest date in the dictionary of articles
        dates = np.array(list(map(get_dates, all_articles)))
        try:
            new_date = np.unique(dates)[0]
        except IndexError:
            new_date = END_DATE_ticker.strftime('%Y-%m-%d')# if there is no more historical data for this stock, just set the 
                                                    # new_date to END_DATE so that the loop goes on to the next stock
        
        # If new_date == DATE, manually adjust the new_date to be the previous date        
        if new_date == DATE:
            new_date = (dt.strptime(new_date, '%Y-%m-%d') - timedelta(days=1)).strftime('%Y-%m-%d')
                
        DATE = new_date


ABBV 0
running for date: 2022-11-05
ACN 1
running for date: 2022-11-05
AEP 2
running for date: 2022-11-05
AIZ 3
running for date: 2022-11-05
ALLE 4
running for date: 2022-11-05
AMAT 5
running for date: 2022-11-05
AMP 6
running for date: 2022-11-05
AMZN 7
running for date: 2022-11-05
AVB 8
running for date: 2022-11-05
AVY 9
running for date: 2022-11-05
AXP 10
running for date: 2022-11-05
BDX 11
running for date: 2022-11-05
BF-B 12
running for date: 2022-11-05
BMY 13
running for date: 2022-11-05
BR 14
running for date: 2022-11-05
CARR 15
running for date: 2022-11-05
CDW 16
running for date: 2022-11-05
CE 17
running for date: 2022-11-05
CHTR 18
running for date: 2022-11-05
CNC 19
running for date: 2022-11-05
CNP 20
running for date: 2022-11-05
COP 21
running for date: 2022-11-05
CTAS 22
running for date: 2022-11-05
CZR 23
running for date: 2022-11-05
DG 24
running for date: 2022-11-05
DPZ 25
running for date: 2022-11-05
DRE 26
running for date: 2022-11-05
DXC 27
running for date: 2022-11-

In [53]:
# Parse the articles into df format
parsed_news = []

for ticker in news.keys():
    for article in news[ticker]:
        date, title, summary = article
        
        # some summaries are missing, then a float is returned, we want to skip those
        if type(summary) != float:
            parsed_news.append([ticker, date, title, summary])
        
columns = ['Ticker', 'Date', 'Title', 'Summary']        
news_df = pd.DataFrame(parsed_news, columns=columns)
news_df['Date'] = pd.to_datetime(news_df['Date'])

In [54]:
# Sentiment Analysis
analyzer = SentimentIntensityAnalyzer()

# Ticker news
scores = news_df['Summary'].apply(analyzer.polarity_scores).tolist()
df_scores = pd.DataFrame(scores)
news_df = news_df.join(df_scores)

# # General market news
# scores = general_market_news['Summary'].apply(analyzer.polarity_scores).tolist()
# df_scores = pd.DataFrame(scores)
# general_market_news = general_market_news.join(df_scores)

In [55]:
news_df.head()

,Ticker,Date,Title,Summary,neg,neu,pos,compound
0,ABBV,2022-11-05,3 No-Brainer Dividend Stocks to Buy in November,There's no need for agonizing decision-making ...,0.328,0.444,0.228,-0.2023
1,ABBV,2022-11-05,3 Reasons to Buy AbbVie Stock Right Now,The drugmaker has something to offer to all ki...,0.000,1.000,0.000,0.0000
2,ABBV,2022-11-04,3 Dividend Kings Crushing the S&P 500 in 2022,Companies in the Dividend King club carry well...,0.000,0.663,0.337,0.8807
3,ABBV,2022-11-04,2 Dividend Growth Stocks to Buy and Hold for Y...,These companies have been raising their divide...,0.000,1.000,0.000,0.0000
4,ABBV,2022-11-04,Better Buy: AbbVie vs. Walgreens Boots Alliance,"Both these companies pay a dividend, but the s...",0.333,0.667,0.000,-0.4588


In [38]:
# Convert Date columns to strings
# general_market_news['Date'] = general_market_news['Date'].dt.date.astype(str)
financial_news['Date'] = financial_news['Date'].dt.date.astype(str)
news_df['Date'] = news_df['Date'].dt.date.astype(str)

# Concat the new articles with the previous ones
# general_market_news = pd.concat([prev_general_market_news, general_market_news])
ticker_news = pd.concat([prev_ticker_news, news_df])
financial_news = pd.concat([prev_financial_news, financial_news])

# Drop duplicates (some stocks might not have many articles and so fetching new articles might also bring some already existing ones)
# general_market_news.drop_duplicates(subset=['Date', 'Headline', 'Summary'], inplace=True)
financial_news.drop_duplicates(subset=['Date', 'Sentiment Score', 'Sentiment Label', 'Topic'], inplace=True)
ticker_news.drop_duplicates(subset=['Ticker', 'Date', 'Title', 'Summary'], inplace=True)

# Sort articles by Ticker, Date
# general_market_news.sort_values(by=['Date'], ascending=False, inplace=True)
financial_news.sort_values(by=['Topic', 'Date'], ascending=False, inplace=True)
ticker_news.sort_values(by=['Ticker', 'Date'], ascending=False, inplace=True)

# Reset index
# general_market_news.reset_index(drop=True, inplace=True)
financial_news.reset_index(drop=True, inplace=True)
ticker_news.reset_index(drop=True, inplace=True)

### Federal Reserve Economic Data

In [82]:
FRED_KEY = '9345824e83618a29590ccf61d6535fda'

In [83]:
categories = {
    'Brent': 'DCOILBRENTEU',
    'WTI': 'DCOILWTICO',
    'Dollar_to_EUR': 'DEXUSEU',
    'SOFR': 'SOFRINDEX',
    'FFD': 'DFF',
    'InflationExp': 'T5YIFR',
    'MoodyCorpBond': 'DAAA',
    'Treasury': 'DGS5',
    'MortgageIndx': 'OBMMIJUMBO30YF',
}

# create an empty dataframe to store the data
dates_timestamps = pd.date_range(start='2021-01-01', end=dt.today(), freq='D').tolist()

FRED = pd.DataFrame(dates_timestamps, columns=['Date'])

In [84]:
start_date = '2021-01-01'
file_type = 'json'

# Loop through the categories and parse the data into a df for each
for category in categories:
    
    series_id = categories[category]
    
    url = 'https://api.stlouisfed.org/fred/series/observations?' + \
          'series_id=' + series_id + \
          '&file_type=' + file_type + \
          '&observation_start=' + start_date + \
          '&api_key=' + FRED_KEY
    
    result = requests.get(url)
    data = result.json()

    # Loop through the observations in the JSON object to parse them into a df
    temp_data_holder = []
    for observation in data['observations']:
        date = observation['date']
        value = observation['value']
        temp_data_holder.append([date, value])

    # Parse the obserations into a df
    df = pd.DataFrame(temp_data_holder, columns=['Date', category])
    df['Date'] = pd.to_datetime(df['Date'])

    # Merge all category data one by one
    FRED = FRED.merge(df, on='Date', how='left')

In [85]:
# Set data as an index
FRED.set_index('Date', inplace=True)

### Iexcloud (essentially no access with free subscription)

In [86]:
# IEX_KEY = 'sk_cc491c649efb431f9a5a06d8e69df9bf'

In [87]:
# url = 'https://cloud.iexapis.com/v1/stock/AAPL/balance-sheet?token=' + IEX_KEY
# result = requests.get(url)
# data = result.json()
# data

In [88]:
# result

### DataFrames:

<li> general_market_news </li>
<li> financial_news </li>
<li> ticker_news </li>

In [89]:
print('general_market_news')
display(general_market_news.head())
display(general_market_news.shape)
print('financial_news')
display(financial_news.head())
display(financial_news.shape)
print('ticker_news')
display(ticker_news.head())
display(ticker_news.shape)
print('FRED')
display(FRED.head())
display(FRED.shape)

general_market_news


,Date,Headline,Summary
0,2022-11-05 14:02:52,Machinists Union Approves Biden Rail Deal on S...,"Nearly 5,000 workers affiliated with the Inter..."
1,2022-11-05 13:01:14,Kosovan Serbs Leave State Jobs in Protest Over...,Serbs in northern Kosovo said they’re quitting...
2,2022-11-05 13:00:34,Biden Calls Protesters ‘Idiots’ for Holding An...,President Joe Biden called protesters holding ...
3,2022-11-05 12:59:00,Key Words: ‘I own the responsibility for why e...,"Twitter founder and former chief executive, J..."
4,2022-11-05 12:57:03,Fox loses legal battle to buy a stake in FanDu...,The decision ends the lawsuit over the valuati...


(100, 3)

financial_news


,Date,Sentiment Score,Sentiment Label,Topic
0,2022-10-29,0.039220,Neutral,financial_market_news
1,2022-10-29,0.372108,Bullish,financial_market_news
2,2022-10-29,0.143791,Neutral,financial_market_news
3,2022-10-29,0.090183,Neutral,financial_market_news
4,2022-10-29,0.248693,Somewhat-Bullish,financial_market_news


...

,Date,Ticker,Title,Summary,neg,neu,pos,compound
0,2022-11-05,XOM,"Amazon, Apple, and Other Hot Names Investors A...",Plus two stocks worth keeping an eye on: SiTim...,0.000,0.863,0.137,0.2263
1,2022-11-05,XOM,"For EVs, solar tax breaks and climate change, ...",Climate may not top inflation in these midterm...,0.043,0.770,0.187,0.6478
2,2022-11-05,XOM,Wall Street Breakfast: What Moved Markets,Listen on the go! A daily podcast of Wall Stre...,0.000,1.000,0.000,0.0000
3,2022-11-04,XOM,This Oil King Stock Will Soon Have Its Crown,The future hasn't looked this bright in years.,0.000,0.675,0.325,0.5228
4,2022-11-03,XOM,Tim Cook has been an excellent leader for Appl...,Apple’s impact on the broader stock market has...,0.000,0.887,0.113,0.4927


(42844, 8)

FRED


,Brent,WTI,Dollar_to_EUR,SOFR,FFD,InflationExp,MoodyCorpBond,Treasury,MortgageIndx
Date,,,,,,,,,
2021-01-01,.,.,.,.,0.09,.,.,.,.
2021-01-02,NaN,NaN,NaN,NaN,0.09,NaN,NaN,NaN,NaN
2021-01-03,NaN,NaN,NaN,NaN,0.09,NaN,NaN,NaN,NaN
2021-01-04,50.37,47.47,1.2254,1.04197497,0.09,2.04,2.28,0.36,2.884
2021-01-05,53.16,49.78,1.2295,1.04197787,0.09,2.05,2.28,0.38,2.871


(674, 9)

### Write the DataFrames to csv

In [90]:
general_market_news.to_csv('Data/general_market_news.csv')
financial_news.to_csv('Data/financial_news.csv')
ticker_news.to_csv('Data/ticker_news.csv')

### Add Technical Indicators

#### MACD

In [91]:
for stock in stocks:
    
    # extract info for 1 stock only
    per_stock_price = prices[stock]
    
    # create a temporary df for the calculations
    temp_df = per_stock_price.copy()

    # use the adjusted close price to calculate short-term (12-day) exponential moving average
    temp_df['st_ema'] = temp_df['Adj Close'].ewm(span = 12.0).mean() 

    # use the adjusted close price to calculate long-term (26-day) exponential moving average
    temp_df['lt_ema'] = temp_df['Adj Close'].ewm(span = 26.0).mean()  

    # calculate the MACD with the formula: MACD = st_ema - lt_ema 
    temp_df['MACD'] = temp_df['st_ema'] - temp_df['lt_ema']

    # calculate MACD's 9-day exponential moving average as its signal line
    temp_df['MACD_signal'] = temp_df['MACD'].ewm(span = 9.0).mean()

    # add MACD_signal column to the prices df 
    prices[stock , 'MACD signal'] = temp_df['MACD_signal']

#### OBV

In [92]:
for stock in stocks:
    
    # extract info for 1 stock only
    per_stock_price = prices[stock]
    
    # create a temporary df for the calculations
    temp_df = per_stock_price.copy()

    # get the price direction
    temp_df['change'] = temp_df['Adj Close'].pct_change()
    temp_df['price_direction'] = temp_df['change'] / temp_df['change'].abs()

    # calculate OBV exponential moving average with center of mass = 5
    temp_df['OBV'] = (temp_df['Volume'] * temp_df['price_direction']).fillna(0).cumsum().astype(int)
    temp_df['OBV ema'] = temp_df['OBV'].ewm(com=5).mean()

    # add OBV ema column to the GOOG_price
    prices[stock, 'OBV ema'] = temp_df['OBV ema']

#### RSI

In [93]:
for stock in stocks:
    
    # extract info for 1 stock only
    per_stock_price = prices[stock]

    # take difference of adj_close
    temp_df['diff'] = temp_df['Adj Close'].diff()

    # add a new column named gain as the positive part of daily price change
    temp_df['gain'] = np.where(temp_df['diff'] >= 0, temp_df['diff'], 0.0)
    temp_df['gain'].iloc[0] = np.nan #because the first values gets assigned 0

    # add a new column named loss as the negative part of daily price change
    temp_df['loss'] = np.where(temp_df['diff'] < 0, temp_df['diff'].abs(), 0.0)
    temp_df['loss'].iloc[0] = np.nan #because the first values gets assigned 0

    # add a new column named avg_gain as the average gain described before
    avg_gain = temp_df['gain'].copy() #just get the gain values
    avg_gain.iloc[14] = temp_df.gain.iloc[0:15].mean() #find the mean of the first 14 days
    avg_gain.iloc[0:14] = np.repeat(np.nan, 14) #fill values before day 14 with nans
    temp_df['avg_gain'] = avg_gain.ewm(alpha = 1/14, adjust = False).mean() #use ewm with 13/14 weight to the previous mean value and 1/14 weight to the new gain value (exactly like in the formula provided to calculate the avg gain)

    # add a new column named avg_loss as the average loss described before
    avg_loss = temp_df['loss'].copy() #same approach here
    avg_loss.iloc[14] = temp_df['loss'].iloc[0:15].mean()
    avg_loss.iloc[0:14] = np.repeat(np.nan, 14) 
    temp_df['avg_loss'] = avg_loss.ewm(alpha = 1/14, adjust = False).mean()

    # add a new column named RSI as the relative strength index described before
    temp_df['RSI'] = 100 - 100/(1+(temp_df['avg_gain'] / temp_df['avg_loss'])) #using the given formula

    # add RSI column to the GOOG_price
    prices[stock, 'RSI'] = temp_df['RSI']

/var/folders/nz/jhg_xmnj7kn3vcf8309msskc0000gp/T/ipykernel_32726/143771241.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['gain'].iloc[0] = np.nan #because the first values gets assigned 0
/var/folders/nz/jhg_xmnj7kn3vcf8309msskc0000gp/T/ipykernel_32726/143771241.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['loss'].iloc[0] = np.nan #because the first values gets assigned 0
/var/folders/nz/jhg_xmnj7kn3vcf8309msskc0000gp/T/ipykernel_32726/143771241.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://

In [94]:
# sort columns index
prices.sort_index(axis=1, inplace=True)

# Data Preprocessing

In [97]:
# Group Sentiment Score per day
# general_market_news_grouped = general_market_news.groupby(by=['Date'])['compound'].mean().to_frame()
financial_news_score = financial_news.groupby(by=['Topic', 'Date'])['Sentiment Score'].mean().to_frame()
financial_news_label = financial_news.groupby(by=['Topic', 'Date'])['Sentiment Label'].agg(pd.Series.mode).to_frame()
financial_news_grouped = financial_news_score.merge(financial_news_label, on=['Topic', 'Date'], how='left')
ticker_news_grouped = ticker_news.groupby(by=['Ticker', 'Date'])['compound'].mean().to_frame()

# Reset indices
# general_market_news_grouped.reset_index(inplace=True)
financial_news_grouped.reset_index(inplace=True)
ticker_news_grouped.reset_index(inplace=True)

In [100]:
# Convert Date column back to timestamp and set as index
financial_news_grouped['Date'] = pd.to_datetime(financial_news_grouped['Date'])
ticker_news_grouped['Date'] = pd.to_datetime(ticker_news_grouped['Date'])

financial_news_grouped.set_index('Date', inplace=True)
ticker_news_grouped.set_index('Date', inplace=True)

In [ ]:
# # Create a list of dates with a daily frequency from 2021-01-01 to today
# date_range = pd.date_range(dt(2021,1,1), dt.today(), freq='D').to_list()
# date_range = list(map(dt.date, date_range))
# data_index = list(map(str, date_range))

# # Create an empty DataFrame which will contain all our feature data
# data = pd.DataFrame({'Date': data_index})

In [245]:
data = prices.copy()
data.head()

ABBV                                                  \
            Adj Close       Close        High         Low MACD signal   
Date                                                                    
2021-01-01        NaN         NaN         NaN         NaN         NaN   
2021-01-02        NaN         NaN         NaN         NaN         NaN   
2021-01-03        NaN         NaN         NaN         NaN         NaN   
2021-01-04  96.813477  105.410004  107.349998  103.860001    0.000000   
2021-01-05  97.814590  106.500000  107.019997  104.629997    0.012478   

                                                            ACN  ...  \
                 OBV ema        Open RSI     Volume   Adj Close  ...   
Date                                                             ...   
2021-01-01  0.000000e+00         NaN NaN        NaN         NaN  ...   
2021-01-02  0.000000e+00         NaN NaN        NaN         NaN  ...   
2021-01-03  0.000000e+00         NaN NaN        NaN         NaN  ...   
2021-01-04  0.000000e+00  107.180000 NaN  9523400.0  249.919205  ...   
2021-01-05  1.901450e+06  105.410004 NaN  6823800.0  251.341995  ...   

                   XOM   XRP-USD                                            \
                Volume Adj Close     Close      High       Low MACD signal   
Date                                                                         
2021-01-01         NaN  0.237444  0.237444  0.249270  0.217288    0.000000   
2021-01-02         NaN  0.221655  0.221655  0.238388  0.215816   -0.000197   
2021-01-03         NaN  0.225848  0.225848  0.237741  0.217167   -0.000248   
2021-01-04  27764700.0  0.236677  0.236677  0.256477  0.219722   -0.000133   
2021-01-05  44035100.0  0.226750  0.226750  0.241251  0.221602   -0.000117   

                                                    
                 OBV ema      Open RSI      Volume  
Date                                                
2021-01-01  0.000000e+00  0.219845 NaN  5888429287  
2021-01-02 -2.603814e+09  0.237670 NaN  4773658725  
2021-01-03 -1.440716e+09  0.221744 NaN  5109898871  
2021-01-04  1.108430e+09  0.226038 NaN  6141918380  
2021-01-05  1.105988e+09  0.236636 NaN  5378491929  

[5 rows x 990 columns]

#### Merge all dataframes together

In [246]:
# # General Market News (not adding anymore as most dates are NULL)
# general_market_news_grouped.columns = ['Date', 'general_market_news_compound']
# data = data.merge(general_market_news_grouped, on=['Date'], how='left')

In [247]:
# Financial News
unique_topics = financial_news_grouped['Topic'].unique()

for stock in stocks:
    
    for topic in unique_topics:

        # Get a df for one topic at a time
        topic_news = financial_news_grouped.loc[financial_news_grouped['Topic'] == topic]
        
        # Rename the columns
        topic_news = topic_news.rename(columns={'Sentiment Score': f'{topic}_sentiment_score', 'Sentiment Label': f'{topic}_sentiment_label'})
        topic_news.drop(columns=['Topic'], inplace=True)

        # Convert columns to multiindex
        topic_news.columns = pd.MultiIndex.from_tuples([(stock, col) for col in topic_news.columns])

        # Join with the data DataFrame
        data = data.merge(topic_news, left_index=True, right_index=True, how='left')

In [248]:
# Financial News

for stock in stocks:

    # Get a df for one topic at a time
    per_ticker_news = ticker_news_grouped.loc[ticker_news_grouped['Ticker'] == stock]

    # Rename the columns
    per_ticker_news = per_ticker_news.rename(columns={'compound': 'news_sentiment'})
    per_ticker_news.drop(columns=['Ticker'], inplace=True)

    # Convert columns to multiindex
    per_ticker_news.columns = pd.MultiIndex.from_tuples([(stock, col) for col in per_ticker_news.columns])


    # Join with the data DataFrame
    data = data.merge(per_ticker_news, left_index=True, right_index=True, how='left')

In [249]:
# Forward fill missing values
data = data.ffill(axis=0)

# Keep only trading days
data = data.loc[data[('ABBV', 'Adj Close')].notna()]

In [253]:
# Convert prices to returns

for stock in stocks:
    data[stock, 'Adj Close'] = data[stock, 'Adj Close'].pct_change()

In [255]:
data.sort_index(axis=1, inplace=True)

In [256]:
data.tail()

ABBV                                                  \
           Adj Close       Close        High         Low MACD signal   
Date                                                                   
2022-10-31 -0.008197  146.399994  146.779999  144.080002    2.456979   
2022-11-01  0.003484  146.910004  147.699997  144.550003    2.353645   
2022-11-02 -0.016268  144.520004  148.080002  144.479996    2.165616   
2022-11-03 -0.000692  144.419998  145.419998  143.029999    1.936586   
2022-11-04  0.005955  145.279999  145.580002  143.080002    1.710743   

                                                            \
                 OBV ema        Open        RSI     Volume   
Date                                                         
2022-10-31  2.770415e+08  145.169998  70.042307  8604800.0   
2022-11-01  2.768466e+08  146.660004  62.932017  6351200.0   
2022-11-02  2.757934e+08  147.029999  55.634287  5345000.0   
2022-11-03  2.742740e+08  144.000000  56.687451  3850700.0   
2022-11-04  2.737784e+08  145.350006  64.771702  4623700.0   

                                                ...     XRP-USD  \
           economy_fiscal_news_sentiment_label  ...      Volume   
Date                                            ...               
2022-10-31                             Neutral  ...  1462007947   
2022-11-01                             Neutral  ...  1273429763   
2022-11-02                             Neutral  ...  1381853436   
2022-11-03                             Neutral  ...  1086924485   
2022-11-04                             Neutral  ...  2774226120   

                                                \
           economy_fiscal_news_sentiment_label   
Date                                             
2022-10-31                             Neutral   
2022-11-01                             Neutral   
2022-11-02                             Neutral   
2022-11-03                             Neutral   
2022-11-04                             Neutral   

                                                \
           economy_fiscal_news_sentiment_score   
Date                                             
2022-10-31                            0.021981   
2022-11-01                            0.021981   
2022-11-02                            0.021981   
2022-11-03                            0.021981   
2022-11-04                            0.021981   

                                               \
           economy_macro_news_sentiment_label   
Date                                            
2022-10-31                            Neutral   
2022-11-01                            Neutral   
2022-11-02                            Neutral   
2022-11-03                            Neutral   
2022-11-04                            Neutral   

                                               \
           economy_macro_news_sentiment_score   
Date                                            
2022-10-31                           0.063556   
2022-11-01                           0.063556   
2022-11-02                           0.063556   
2022-11-03                           0.063556   
2022-11-04                           0.063556   

                                                  \
           economy_monetary_news_sentiment_label   
Date                                               
2022-10-31                               Neutral   
2022-11-01                               Neutral   
2022-11-02                               Neutral   
2022-11-03                               Neutral   
2022-11-04                               Neutral   

                                                  \
           economy_monetary_news_sentiment_score   
Date                                               
2022-10-31                              0.064489   
2022-11-01                              0.064489   
2022-11-02                              0.064489   
2022-11-03                              0.064489   
2022-11-04                          

### Write data in a single file

In [257]:
data.to_csv('Data/data.csv')

### Scrape Options Data From Yahoo Finance

In [336]:
put_call_ratios = {}

headers = { 
    'User-Agent'      : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36', 
    'Accept'          : 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 
    'Accept-Language' : 'en-US,en;q=0.5',
    'DNT'             : '1', 
    'Connection'      : 'close'
}


for stock in stocks[:100]:
    
    # Fetch the data
    url = f'https://finance.yahoo.com/quote/{stock}/options?p={stock}'
    response = requests.get(url, headers=headers, timeout=5)
    
    # Find the table with options data
    soup = BeautifulSoup(response.text)
    data = soup.find_all('table')
    
    try:
        # Read call, put options data into df
        calls = pd.read_html(str(data[0]))[0]
        puts = pd.read_html(str(data[1]))[0]

        # Format the df
        calls.replace({'-': '0'}, inplace=True)
        puts.replace({'-': '0'}, inplace=True)

        # Get the total volume for call, put options
        calls['Volume'] = calls['Volume'].astype(int)
        puts['Volume'] = puts['Volume'].astype(int)

        # Calculate the ratio and write it down to the dictionary
        ratio = puts['Volume'].sum()/calls['Volume'].sum()
        put_call_ratios[stock] = ratio
        
    except IndexError:
        
        # If there is no options data, just use np.nan
        put_call_ratios[stock] = np.nan
        
    
    time.sleep(30)

/var/folders/nz/jhg_xmnj7kn3vcf8309msskc0000gp/T/ipykernel_32726/10864136.py:31: RuntimeWarning: divide by zero encountered in long_scalars
  ratio = puts['Volume'].sum()/calls['Volume'].sum()


In [338]:
options_data = pd.DataFrame.from_dict(put_call_ratios, orient='index')
options_data.to_csv('Data/options_data.csv')